In [9]:
from Servicios.GoogleDriveService.GoogleDriveService import GoogleDriveService
from Servicios.OrquestadorService.OrchestrateService import OrchestrateService
from IPython.display import clear_output

In [ ]:
# Crear variables
IdContenedorBronze = "1yQCQ3BhNPdXiz890KTF9iNB5qdeTATlV"
IdContenedorGold = "1TMq41pVhsEDK1IUP4kQHzf7aIsagVihE"

In [20]:

# Crear instancia de GoogleDriveService
google_drive_service = GoogleDriveService()

# Crear instancia de OrchestrateService, inyectando GoogleDriveService
orchestrate_service = OrchestrateService(google_drive_service)

# Ejecutar el método gdriveservice_listar_archivos desde OrchestrateService
orchestrate_service.gdriveservice_listar_archivos()
print(8)

hola
hola2
hola3
hola4
hola5


AttributeError: 'OrchestrateService' object has no attribute 'gdriveservice_listar_archivos'